In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0781ew_0
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0781ew_0
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.1.0-py3-none-any.whl size=8011 sha256=e2b2acbb79d726227ace2ffdeb18b901e4debb9cf9419b13a35a389a655ee4ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-pwklxwej/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [3]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmpnuqnrc0e".


**Question1 (Q1)**

In [10]:

%%writefile Q1_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
using namespace std;

// Kernel definition
__global__ void ArrAdd(double *arr, int size,double *result)
{
    // declares a shared memory segment that is accessible by all threads in the same block. More on this later.
extern __shared__ double partialSum[];


unsigned int t = threadIdx.x;

unsigned int new_size = (size-1) / blockDim.x +1 ;
partialSum[t] = 0;
for(int i=0;i<new_size ;i++)
{

partialSum[t]+= arr[t*new_size+i];

}
//partialSum[t] = arr[t];

if(t >=size)
   return ;
for(unsigned int stride = 1; stride < blockDim.x; stride *= 2){

     if(t % (2*stride) == 0)
          partialSum[t] += partialSum[t+stride];
     __syncthreads();
}

 if(t==0)
     *result = partialSum[0];

}

int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

 double* array = nullptr;  // Pointer to the array
 double * result = nullptr;
    double value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        double* temp = static_cast<double*>(realloc(array, (count + 1) * sizeof(double)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();

    // Print the elements in the array
    for (int i = 0; i < count; i++) {
        std::cout << array[i] << std::endl;
    }



    // host to device allocation
     double *array_device;
     double *result_device;

     result = (double*)malloc(sizeof(double));
     cudaMalloc((void**)&array_device, count * sizeof(double));
     cudaMalloc((void**)&result_device,sizeof(double));

    cudaMemcpy(array_device, array, count * sizeof(double), cudaMemcpyHostToDevice);
    dim3 threadsPerBlock(1024, 1);
    dim3 numBlocks(1, 1);
    ArrAdd<<< numBlocks, threadsPerBlock , threadsPerBlock.x * sizeof(double) >>>(array_device,count,result_device);

    cudaMemcpy( result, result_device, sizeof(double), cudaMemcpyDeviceToHost);

    //cout<<"final sum is: "<<*result<<endl;
    cout << fixed <<setprecision(2) << *result <<endl;

    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(array);

   return 0;
}


Overwriting Q1_2_31_2_33.cu


In [11]:
!chmod +x Q1_2_31_2_33.cu
!nvcc Q1_2_31_2_33.cu -o Q1
!./Q1 "input_million0.txt"

Streaming output truncated to the last 5000 lines.
995002
995003
995004
995005
995006
995007
995008
995009
995010
995011
995012
995013
995014
995015
995016
995017
995018
995019
995020
995021
995022
995023
995024
995025
995026
995027
995028
995029
995030
995031
995032
995033
995034
995035
995036
995037
995038
995039
995040
995041
995042
995043
995044
995045
995046
995047
995048
995049
995050
995051
995052
995053
995054
995055
995056
995057
995058
995059
995060
995061
995062
995063
995064
995065
995066
995067
995068
995069
995070
995071
995072
995073
995074
995075
995076
995077
995078
995079
995080
995081
995082
995083
995084
995085
995086
995087
995088
995089
995090
995091
995092
995093
995094
995095
995096
995097
995098
995099
995100
995101
995102
995103
995104
995105
995106
995107
995108
995109
995110
995111
995112
995113
995114
995115
995116
995117
995118
995119
995120
995121
995122
995123
995124
995125
995126
995127
995128
995129
995130
995131
995132
995133
995134
995135
995136
9951

**Question2 (Q2)**

In [6]:

%%writefile Q2_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>

using namespace std;

#define block_dim 512
#define search_interval 16
#define shared_size block_dim*search_interval

// Kernel definition
__global__ void BinarySearch(float *array, int size, float target, float *index, bool* found)
{

    *index = -1;

    if(*found == false) { // not found yet

        int thread_id = threadIdx.x;
        __shared__ float shared_array[shared_size];


        for(int i = 0; i < search_interval; i++) {
            int shared_location = thread_id * search_interval + i;
          //  int location = shared_location + shared_size * blockIdx.x;
          int location = shared_location;


            if(location < size) {
              shared_array[shared_location] = array[location];
            }
            else {
              shared_array[shared_location] = INT_MAX;
            }
        }
        __syncthreads();

        int start = 0;
        int end = search_interval - 1;
        int mid = (start + end) / 2;

        float *thread_part = &shared_array[thread_id * search_interval];

        while(start <= end && *found == false)
        {
            if(thread_part[mid] < target) {
                start = mid + 1;
            }
            else if(thread_part[mid] > target) {
                end = mid - 1;
            }
            else {
                *found = true;
               // *index = mid + thread_id * search_interval + shared_size * blockIdx.x;
               *index = mid + thread_id * search_interval ;

            }

            mid = (start + end) / 2;
        }

    }
}


int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];
    float target = atof(argv[2]);

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

    float* array = nullptr;  // Pointer to the array
    float * result = nullptr;
    float value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        float* temp = static_cast<float*>(realloc(array, (count + 1) * sizeof(float)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();

    // Print the elements in the array
    for (int i = 0; i < count; i++) {
        std::cout << array[i] << std::endl;
    }


      bool * flag;
    // host to device allocation
     float *array_device;
     float *result_device;
      bool * flag_device;

     result = (float*)malloc(sizeof(float));
     flag = (bool*)malloc(sizeof(bool));
     *flag = false;

     cudaMalloc((void**)&array_device, count * sizeof(float));
     cudaMalloc((void**)&result_device,sizeof(float));
     cudaMalloc((void**)&flag_device,sizeof(bool));

    cudaMemcpy(array_device, array, count * sizeof(float), cudaMemcpyHostToDevice);
        cudaMemcpy(flag_device, flag, sizeof(bool), cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(512, 1);
    dim3 numBlocks(1,1);
    BinarySearch<<< numBlocks, threadsPerBlock , threadsPerBlock.x * sizeof(float) >>>(array_device,count, target,result_device, flag_device);

    cudaMemcpy( result, result_device, sizeof(float), cudaMemcpyDeviceToHost);

    cout<<"target index : "<<*result<<endl;


    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(array);

   return 0;
}


Writing Q2_2_31_2_33.cu


In [7]:
!chmod +x Q2_2_31_2_33.cu
!nvcc Q2_2_31_2_33.cu -o Q2
!./Q2 "input_million0.txt" 552

Streaming output truncated to the last 5000 lines.
530401
530402
530403
530404
530405
530406
530407
530408
530409
530410
530411
530412
530413
530414
530415
530416
530417
530418
530419
530420
530421
530422
530423
530424
530425
530426
530427
530428
530429
530430
530431
530432
530433
530434
530435
530436
530437
530438
530439
530440
530441
530442
530443
530444
530445
530446
530447
530448
530449
530450
530451
530452
530453
530454
530455
530456
530457
530458
530459
530460
530461
530462
530463
530464
530465
530466
530467
530468
530469
530470
530471
530472
530473
530474
530475
530476
530477
530478
530479
530480
530481
530482
530483
530484
530485
530486
530487
530488
530489
530490
530491
530492
530493
530494
530495
530496
530497
530498
530499
530500
530501
530502
530503
530504
530505
530506
530507
530508
530509
530510
530511
530512
530513
530514
530515
530516
530517
530518
530519
530520
530521
530522
530523
530524
530525
530526
530527
530528
530529
530530
530531
530532
530533
530534
530535
5305

In [8]:
!nvprof ./Q2 "input_million0.txt" 1222

Streaming output truncated to the last 5000 lines.
635278
635279
635280
635281
635282
635283
635284
635285
635286
635287
635288
635289
635290
635291
635292
635293
635294
635295
635296
635297
635298
635299
635300
635301
635302
635303
635304
635305
635306
635307
635308
635309
635310
635311
635312
635313
635314
635315
635316
635317
635318
635319
635320
635321
635322
635323
635324
635325
635326
635327
635328
635329
635330
635331
635332
635333
635334
635335
635336
635337
635338
635339
635340
635341
635342
635343
635344
635345
635346
635347
635348
635349
635350
635351
635352
635353
635354
635355
635356
635357
635358
635359
635360
635361
635362
635363
635364
635365
635366
635367
635368
635369
635370
635371
635372
635373
635374
635375
635376
635377
635378
635379
635380
635381
635382
635383
635384
635385
635386
635387
635388
635389
635390
635391
635392
635393
635394
635395
635396
635397
635398
635399
635400
635401
635402
635403
635404
635405
635406
635407
635408
635409
635410
635411
635412
6354